# Test models

## Test scoring model

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import pickle
import numpy as np
import pandas as pd

from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support

import torch
from transformers import BertTokenizer
from transformers import BertForSequenceClassification

data_path='../../DATASETS/IT_TEXTS'

Xy = pd.read_feather(f'{data_path}/PREPROCESSING/Xy.feather')
Xy.fillna('', inplace=True)
#X, y = Xy[['title', 'summary']], Xy['class']
#del Xy
#
train, test = train_test_split(Xy, test_size=.2, random_state=0)

<ipython-input-1-89aa33382ac8>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm_notebook().pandas()


In [30]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertForSequenceClassification.from_pretrained(f'{data_path}/MODELS/checkpoint-80000')

max_length = 96

def classificate(x):
    t, s = x.title, x.summary
    tokens = tokenizer.encode_plus(t, s,
                                   truncation=True,
                                   padding=True,
                                   max_length=max_length,
                                   return_tensors="pt")
    classification_logits = model(**tokens)[0]
    results = torch.softmax(classification_logits, dim=1).tolist()[0]
    return results[1]

In [31]:
_ = test[:1000]
_['score'] = _.progress_apply(classificate, axis=1)

<ipython-input-31-2b1f06663c34>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _['score'] = _.progress_apply(classificate, axis=1)


In [32]:
_['round_score'] = _['score'].apply(round)

<ipython-input-32-dbc860c0d87d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _['round_score'] = _['score'].apply(round)


In [33]:
precision_recall_fscore_support(_['class'], _['round_score'], average='macro')

(0.6571246164301748, 0.6435924943067397, 0.6495388324075044, None)

checkpoint | F1
---|---
10000 | 0.674
13000 | 0.691
29000 | 0.682
50000 | 0.675
60000 | 0.675
64000 | 0.668
66000 | 0.672
68000 | 0.667
70000 | 0.693
80000 | 0.649